In [1]:
from mayavi import mlab
from ase.io.cube import read_cube_data
import numpy as np
import io2

mlab.init_notebook()
mlab.figure()
data, atoms = read_cube_data('ch4_1.cube')
b2a = io2.Units().b2a
cell = np.array([8.46, 8.46, 8.46]) * b2a
orig = np.array([-4.189227,   -4.189227,   -4.189227])
#grids = np.array(data.shape)

Notebook initialized with ipy backend.


In [2]:
#mlab.figure()
#mlab.test_plot3d()
#mlab.axes(x_axis_visibility=True,y_axis_visibility=True)

In [3]:
import io2.visual as iov

In [4]:
#iov.contour3d([0.02,-0.02], ['red','green'], filename='ch4_1.cube')

In [5]:
#iov.contour3d([0.02,-0.02], ['red','green'], filename='ch4_2.cube')

In [83]:
from pyscf.lo.orth import pre_orth_ao_atm_scf, lowdin
import numpy as np
import numpy.linalg as LA
from pyscf import gto, scf
from aqml.cheminfo.base import *
from functools import reduce
import io2.visual as iov
from ase import Atoms
import interfaces._pyscf as pscf

lmap = {'s':0, 'p':1, 'd':2, 'f':3, 'g':4}
mmap = {'':0, \
        'x':0, 'y':1, 'z':2, \
        'xy':0, 'yz':1, 'z^2':2, 'xz':3, 'x2-y2':4 }
nmap = {1:1, 6:2, 7:2, 8:2, 9:2, 15:3,16:3,17:3} # for MINBASIS


class pyscf_object(object):

    def __init__(self, symbs, coords, basis):
        zs = [ chemical_symbols.index(si) for si in symbs ]
        na = len(symbs)
        symbs1 = [ symbs[0], ] + [ 'H@2' ]*(na-1)
        basis1 = { symbs[0]: basis, 'H@2':'sto-3g' }
        spin1 = sum(zs)%2
        str_m = '' #'O 0 0 0; H 0 0 1; H 0 1 0'
        for i, si in enumerate(symbs1):
            x, y, z = coords[i]
            str_m += '%s %.8f %.8f %.8f; '%(si, x, y, z)
        str_m = str_m[:-2]
        mol = gto.M(atom=str_m, basis=basis1, verbose=0, spin=spin1)
        self.mol = mol
        self.atoms = Atoms(symbs,coords)
        self.na = na
        self.nbf = mol.nao
        self.zs = zs 
        
    def get_ao_map(self):
        ids = self.mol.offset_ao_by_atom()[:, 2:4]
        ibs, ies = ids[:,0], ids[:,1]
        self.ibs = ibs
        self.ies = ies
        idx = [ np.arange(ibs[i],ies[i]) for i in range(self.na) ]
        self.idx = idx
        imap = np.zeros(self.nbf).astype(np.int)
        for i in range(self.na):
            imap[ibs[i]:ies[i]] = i
        self.imap = imap
        labels = self.mol.ao_labels()
        nlms = []
        for _lb in labels:
            lb = _lb.strip().split()[-1]
            n = int(lb[0])
            l = lmap[lb[1]]
            m = mmap[lb[2:]]
            nlms.append( [n,l,m] )
        nlms = np.array(nlms, np.int)
        
        _idxc = idx[0] # AO idx of the central atom
        _iaosr = np.arange(len(_idxc))
        _nlms = nlms[_idxc]

         # to be changed if a bsis other than minimal bst is used
        #flt = np.logical_and(_nlms[:,0]==nmap[self.zs[0]], _nlms[:,1]>0)
        flt = ( _nlms[:,0]==nmap[self.zs[0]] )
        iaosr = _iaosr[flt]

        idxc = _idxc[ flt ] # AO's to be rotated
        #idxc_f = _idxc[ _nlms[:,1]==0 ] # AO not to be rotated

        idxs = np.arange(self.nbf)
        idxl = np.setdiff1d(idxs, _idxc) # AO idx of ligands (i.e., H's)
        self.idxl = idxl
        self.idxc = idxc

    def get_hao(self):
        """
        C: central atom
        L: ligands (i.e., hydrogen atoms)
        """
        mol = self.mol
        # get atomic orbitals (from RHF calculations of free atoms)
        T0 = pre_orth_ao_atm_scf(mol) #
        self.T0 = T0
        _s = mol.intor_symmetric('int1e_ovlp')
        s = reduce( np.dot, (T0.conjugate().T, _s, T0) )
        #print( ' s1 = ', reduce( np.dot, (T0, _s, T0.conjugate().T) ))
        #print( ' s2 = ', s )
        self.get_ao_map()
        s1 = s[self.idxl][:,self.idxc] 
        #''' new basis is |mu> c^{lowdin}_{mu i} '''        
        #c_orth = np.dot(c, lowdin(s1))
        U, eigs, VT = LA.svd(s1, full_matrices=False, compute_uv=True)
        R = np.dot(U,VT)
        T1 = np.eye(self.nbf)
        ib,ie = self.idxc[0], self.idxc[-1]+1
        T1[ib:ie,ib:ie] = R #.T
        self.T1 = T1
        self.coeffs = np.dot(T0,T1)
        return eigs, R #.T

    def visualize_hao(self, grids=[80,80,80]):
        oobj = pscf.io(self.mol)
        origin, cell, data = oobj.orbital(self.coeffs, grids=grids)
        return self.atoms, origin, cell, data

In [84]:
np.set_printoptions(precision=2,suppress=True)

In [85]:
symbols, coords = read_xyz_simple('ch4.xyz')[:2]
obj = pyscf_object(symbols, coords, 'sto-3g')
eigs, c = obj.get_hao()

In [86]:
print( 'eigs = ', eigs)
print( ' c = \n', c)

eigs =  [0.98 0.54 0.54 0.54]
 c = 
 [[ 0.5 -0.5 -0.5 -0.5]
 [ 0.5  0.5  0.5 -0.5]
 [ 0.5  0.5 -0.5  0.5]
 [ 0.5 -0.5  0.5  0.5]]


In [89]:
atoms, origin, cell, dt = obj.visualize_hao(grids=[80,80,80])
isovalues=[-0.1,0.1]; colors=['green','red']
ihao = 0
vobj = iov.contour3d(isovalues, colors, fid=1, atoms=atoms, data=dt[ihao], box=cell, origin=origin)
vobj

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01^\x00\x00\x01^\x08\x02\x00\x00\x00BI\xe0M\x00\x00…

In [49]:
obj.coeffs

array([[ 0.99, -0.14, -0.14, -0.14, -0.14,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.02,  0.52,  0.52,  0.52,  0.52,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  , -0.5 ,  0.5 ,  0.5 , -0.5 ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  , -0.5 ,  0.5 , -0.5 ,  0.5 ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  , -0.5 , -0.5 ,  0.5 ,  0.5 ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  1.  ]])

In [54]:
obj.T0

array([[ 0.99, -0.28,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.02,  1.03,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  1.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  1.  ]])

In [55]:
obj.T1

array([[ 1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0.5,  0.5,  0.5,  0.5,  0. ,  0. ,  0. ,  0. ],
       [ 0. , -0.5,  0.5,  0.5, -0.5,  0. ,  0. ,  0. ,  0. ],
       [ 0. , -0.5,  0.5, -0.5,  0.5,  0. ,  0. ,  0. ,  0. ],
       [ 0. , -0.5, -0.5,  0.5,  0.5,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ]])

In [12]:
symbols, coords = read_xyz_simple('test/c02h06.xyz')[:2]
obj = pyscf_object(symbols, coords, 'sto-3g')
eigs, c = obj.get_hao()

RuntimeError: Electron number 13 and spin 0 are not consistent
Note mol.spin = 2S = Nalpha - Nbeta, not 2S+1

In [ ]:
eigs